In [1]:
import pandas as pd
from pandas import Series,DataFrame

"""
IMPORTANT NOTE: If parsing Excel files fails, conda install xlrd AND openpyxl
"""

# BLS dataset for labor participation
bls_file = pd.ExcelFile('staadata.xlsx')
# Parse the first sheet of the excel file and set as DataFrame
header = ['FIPS Code', 'State', 'Year', 'Civilian non-institutional population',
           'Civilian labor force Total', 'CLF Percent of population', 'CLF Employment Total',
           'CLF Employment Percent of population', 'CLF Unemployment Total',
           'CLF Unemployment Rate'
          ]
bls = bls_file.parse('Sheet1', skiprows=7, names=header)
bls12 = bls[bls.Year == 2012]
bls12 = bls12[['State','CLF Employment Percent of population','CLF Unemployment Rate']]
bls12['CLF Employment Percent of population'] = pd.to_numeric(bls12['CLF Employment Percent of population'])
bls12['CLF Unemployment Rate'] = pd.to_numeric(bls12['CLF Unemployment Rate'])
bls12.set_index('State')

# 2012 Election data to determine if a state is "red" or "blue"
pres_file = pd.ExcelFile('2012pres.xls')
pres = pres_file.parse('2012 Pres General Results')
pres = pres.drop([1, 'FEC ID', 'STATE ABBREVIATION', 'GENERAL ELECTION DATE'], axis=1)
winner = pres[pres['WINNER INDICATOR'] == 'W']
winner = winner[['STATE','LAST NAME', 'GENERAL %']]
winner.rename(columns={'LAST NAME': 'Winner', 'GENERAL %': '%vote won'}, inplace=True)

# Load income per capita as response variable
income = pd.read_csv('per_capita_income.csv', low_memory=False)
income = income[[1, 2]]
income.set_index('State')

# Merge data sources
data = pd.merge(winner, income, left_on='STATE', right_on='State')
data = pd.merge(data, bls12).drop(['STATE'], axis=1).set_index('State')
data.columns = ['Winner', '%VoteWon', 'IncomePerCapita', 'LaborParticipation%', 'Unemployment%']
data['%VoteWon'] = pd.to_numeric(data['%VoteWon'])
data['IncomePerCapita'] = data['IncomePerCapita'].apply(lambda x: x.replace('$', '').replace(',', ''))
data['IncomePerCapita'] = pd.to_numeric(data['IncomePerCapita'])
data['LaborParticipation%'] = pd.to_numeric(data['LaborParticipation%'])
data['Unemployment%'] = pd.to_numeric(data['Unemployment%'])
data

,Winner,%VoteWon,IncomePerCapita,LaborParticipation%,Unemployment%
State,,,,,
Alabama,Romney,0.605458,23606,53.4,8.0
Alaska,Romney,0.548016,33062,63.4,7.1
Arizona,Romney,0.536545,25715,55.5,8.3
Arkansas,Romney,0.605669,22883,54.8,7.6
California,Obama,0.602390,30441,56.6,10.4
Colorado,Obama,0.514921,32357,63.6,7.9
Connecticut,Obama,0.580568,39373,60.9,8.3
Delaware,Obama,0.586064,30488,57.4,7.2
District of Columbia,Obama,0.909131,45877,62.9,9.0


In [2]:
# See the count, mean, standard deviation
data.groupby('Winner').describe()

%VoteWon  IncomePerCapita  LaborParticipation%  Unemployment%
Winner                                                                      
Obama  count  27.000000        27.000000            27.000000      27.000000
       mean    0.577985     31342.185185            60.137037       7.755556
       std     0.084465      4805.524842             3.598199       1.635033
       min     0.500079     23683.000000            54.200000       5.000000
       25%     0.521721     28095.500000            57.400000       6.850000
       50%     0.561634     30441.000000            59.600000       7.800000
       75%     0.604451     33573.500000            63.000000       8.900000
       max     0.909131     45877.000000            66.700000      11.200000
Romney count  24.000000        24.000000            24.000000      24.000000
       mean    0.588346     25911.458333            59.425000       6.941667
       std     0.053165      2867.340610             5.333915       1.727506
       min     0.503931     21036.000000            50.200000       3.100000
       25%     0.547415     24431.500000            55.425000       5.625000
       50%     0.581057     25422.000000            58.600000       7.150000
       75%     0.605511     27000.500000            63.575000       8.225000
       max     0.727905     33071.000000            70.800000       9.300000

In [3]:
# Calculating the F-statistic and associated p value
import statsmodels.formula.api as smf
model1 = smf.ols(formula='IncomePerCapita ~ C(Winner)', data=data)
results1 = model1.fit()
print (results1.summary())

                            OLS Regression Results                            
Dep. Variable:        IncomePerCapita   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     23.26
Date:                Fri, 05 Aug 2016   Prob (F-statistic):           1.42e-05
Time:                        09:58:32   Log-Likelihood:                -494.52
No. Observations:                  51   AIC:                             993.0
Df Residuals:                      49   BIC:                             996.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
Intercept            3.134e+04    

P-value is less than .05 so **vote Democrats** if you want to make more money_(assuming there is causal relationship)_!